In [27]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED, DATABASE_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file, TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [ ]:
N = 30

test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST_600" / "index.json")
number_of_instances = N // 5

instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TRAIN_200" / "index.json")
train_instances_200 = InstanceList()
for i in range(5):
    train_instances_200.extend(instances[i * 200 : i * 200 + number_of_instances])

instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json")
train_instances_400 = InstanceList()
for i in range(5):
    train_instances_400.extend(instances[i * 200 : i * 200 + number_of_instances])

instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TRAIN_600" / "index.json")
train_instances_600 = InstanceList()
for i in range(5):
    train_instances_600.extend(instances[i * 200 : i * 200 + number_of_instances])

portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=2)

aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances_200,
    prefix="config",
    max_iter=75,
    calculate_features=False,
    estimator=None,
)
portfolio = aac.configure()

aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances_400,
    prefix="config",
    max_iter=75,
    calculate_features=False,
    estimator=None,
)
portfolio = aac.configure()

aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances_600,
    prefix="config",
    max_iter=75,
    calculate_features=False,
    estimator=None,
)
portfolio = aac.configure()

[2025-02-19 22:19:39] DEBUG     AAC(prefix=config, iter=1, configuration_time=[inf inf])
[2025-02-19 22:19:39] DEBUG     Portfolio.evaluate(config;aac_iter=1)
[2025-02-19 22:19:39] DEBUG     Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]
[2025-02-19 22:19:39] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN_200/cluster_netgen/000.tsp))
[2025-02-19 22:19:39] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/TRAIN_200/cluster_netgen/000.tsp))
[2025-02-19 22:19:39] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN_200/compression/000.tsp))
[2025-02-19 22:19:39] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/TRAIN_200/compression/000.tsp))
[2025-02-19 22:19:39] DEBUG     solve(prefix=confi

In [ ]:
# import joblib
# paths = list((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "200" / "database").glob("*.db"))
# for path in paths:
#     db = DB(path)

#     configured_portfolio_solvers = pd.read_sql_query(""" 
#     SELECT DISTINCT solver_id FROM results
#     WHERE prefix LIKE 'test%'                  
#     """, db._conn)["solver_id"].to_list()

#     portfolio = Portfolio.from_iterable([TSP_LKH_Solver.from_db(id_, db) for id_ in configured_portfolio_solvers])
#     joblib.dump(portfolio, MAIN_DIR / "portfolios" / "200" / f"{path.stem}.pkl")

# paths = list((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "400" / "database").glob("*.db"))
# for path in paths:
#     db = DB(path)

#     configured_portfolio_solvers = pd.read_sql_query(""" 
#     SELECT DISTINCT solver_id FROM results
#     WHERE prefix LIKE 'test%'                  
#     """, db._conn)["solver_id"].to_list()

#     portfolio = Portfolio.from_iterable([TSP_LKH_Solver.from_db(id_, db) for id_ in configured_portfolio_solvers])
#     joblib.dump(portfolio, MAIN_DIR / "portfolios" / "400" / f"{path.stem}.pkl")

In [1]:
# import joblib

# paths = list((MAIN_DIR / "portfolios" / "200").glob("run-plain-30-*.pkl"))
# for path in paths:
#     run_id = path.stem.split("-")[-1]
#     portfolio = joblib.load(path)

#     for i in range(100):
#         portfolio.evaluate(
#             test_instances,
#             prefix=f"{run_id};test{i}",
#             calculate_features=False,
#             cache=False,
#         )